In [1]:
import os
os.chdir("..")
print(os.getcwd())

/Users/lukas/Library/CloudStorage/OneDrive-StudentsRWTHAachenUniversity/5. Semester/Machine Learning/COMP5212


In [2]:
MODEL_PATH = "checkpoints/model_nrg01.hpc.itc.rwth-aachen.de_20231123-201621_mobilenet_v3_large_0_unfreezed_20_epochs.pth"
TSV_PATH = "data/kaggle_dataset/train.tsv"
TEST_TSV_PATH = "data/kaggle_dataset/test.tsv"
IMAGE_PATH = "notebooks/demo_images/day_winter.png"

In [3]:
from PIL import Image
import torch
from torchvision import transforms
import numpy as np

/Users/andreaspletschko/.local/lib/python3.11/site-packages/requests/__init__.py:102: RequestsDependencyWarning: urllib3 (1.26.18) or chardet (5.2.0)/charset_normalizer (2.0.12) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({})/charset_normalizer ({}) doesn't match a supported "


In [4]:
from src.models.model import get_model
import src.kaggle50k_dataset as kaggle50k_dataset

In [5]:
def load_model(model_path):
    state_dict = torch.load(model_path, map_location=torch.device('cpu'))
    model_state_dict = state_dict["model_state_dict"]
    idx_to_label = state_dict["idx_to_labels"]
    config = state_dict["config"]
    model = get_model(model_name=config["model"]["name"], num_classes=56)
    model.load_state_dict(model_state_dict)
    model.eval()
    return model, idx_to_label, config

def get_idx_to_label(tsv_path):
    labels = []
    tsv_file = open(tsv_path, "r")

    for line in tsv_file:
        if line in ['\n', '\r\n']:
            continue
        label, _ = os.path.split(line.strip())
        labels.append(label)

    idx_to_label = {idx: label for idx, label in enumerate(set(labels))}
    return idx_to_label

def load_img(image_path):
    transform = transforms.Compose([
        transforms.Resize((1536, 662)),
        transforms.ToTensor(),
        transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
    ])

    image = Image.open(image_path).convert("RGB")
    image_transformed = transform(image)
    return image_transformed

def get_random_image(dataset):
    idx = np.random.randint(0, len(dataset))
    image, label = dataset[idx]
    return image, label

In [7]:
print(os.getcwd())

model, idx_to_label, _ = load_model(MODEL_PATH)
image = load_img(IMAGE_PATH)
image = image.unsqueeze(0)

with torch.no_grad():
    output = model(image)
    # top 3
    _, predicted = torch.topk(output.data, 50, dim=1)
    predicted = predicted.squeeze(0)
    print(predicted)
    for idx in predicted:
        print(idx)
        print(idx_to_label[idx.item()])
    print(idx_to_label[predicted[0].item()])
    print(idx_to_label[predicted[1].item()])
    print(idx_to_label[predicted[2].item()])


/Users/lukas/Library/CloudStorage/OneDrive-StudentsRWTHAachenUniversity/5. Semester/Machine Learning/COMP5212
dict_keys(['model_state_dict', 'idx_to_labels', 'config'])
MobileNetV3(
  (features): Sequential(
    (0): Conv2dNormActivation(
      (0): Conv2d(3, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(16, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
      (2): Hardswish()
    )
    (1): InvertedResidual(
      (block): Sequential(
        (0): Conv2dNormActivation(
          (0): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=16, bias=False)
          (1): BatchNorm2d(16, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
          (2): ReLU(inplace=True)
        )
        (1): Conv2dNormActivation(
          (0): Conv2d(16, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (1): BatchNorm2d(16, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
        )
 